# Data prep

## In this notebook we will load the songs from the zip file, and perform transoformations to prepare the data for two-tower training
Steps
1. Extract from the zip file
2. Upload to BQ
3. Enrich features for the playlist songs
4. Cross-join songs with features (excpected rows = n_songs x n_playlists)
5. Remove after-the-fact (later position songs) from the newly generated samples
6. Create a clean train table, and flatten structs or use arrays

#### Unzip the file and uplaod to BQ

In [ ]:
# !gsutil cp gs://matching-engine-content/spotify-million-playlist/spotify_million_playlist_dataset.zip .
# !unzip spotify_million_playlist_dataset.zip

In [ ]:
# import os
# import json

# data_files = os.listdir('data')
# # PROJECT_ID = 'jtotten-project'
# bq_dataset = 'mdp_eda'

# for filename in data_files:
#   with open(f'data/{filename}') as f:
#     print(f)
#     json_dict = json.load(f)
#     df = pd.DataFrame(json_dict['playlists'])
#     df.to_gbq(
#     destination_table=f'{bq_dataset}.playlists', 
#     project_id=f'{PROJECT_ID}', # TODO: param
#     location='us-central1', 
#     progress_bar=True, 
#     reauth=True, 
#     if_exists='append'
#     ) 

### Need to find playlist_nested lineage

Here's the ddl from the schema - could be created from playlists table below

```
'CREATE TABLE `hybrid-vertex.spotify_mpd.playlists_nested`\n(\n  name STRING,\n  collaborative STRING,\n  pid INT64,\n  modified_at INT64,\n  num_tracks INT64,\n  num_albums INT64,\n  num_followers INT64,\n  num_edits INT64,\n  duration_ms INT64,\n  num_artists INT64,\n  description STRING,\n  tracks ARRAY<STRUCT<pos STRING, artist_name STRING, track_uri STRING, artist_uri STRING, track_name STRING, album_uri STRING, duration_ms STRING, album_name STRING>>,\n  row_number INT64\n);'
```

In [ ]:
# %%bigquery
# create or replace table jtotten-project.spotify_mpd.playlists as (
# with json_parsed as (SELECT *, JSON_EXTRACT_ARRAY(tracks) as json_data FROM `jtotten-project.mdp_eda.playlists` )

# select json_parsed.* except(tracks, duration_ms, json_data),
# JSON_EXTRACT(jsn_line, "$.pos") as pos, 
# JSON_EXTRACT(jsn_line, "$.artist_name") as artist_name,
# JSON_EXTRACT(jsn_line, "$.track_uri") as track_uri,
# JSON_EXTRACT(jsn_line, "$.artist_uri") as artist_uri,
# JSON_EXTRACT(jsn_line, "$.track_name") as track_name,
# JSON_EXTRACT(jsn_line, "$.album_uri") as album_uri,
# JSON_EXTRACT(jsn_line, "$.duration_ms") as duration_ms,
# JSON_EXTRACT(jsn_line, "$.album_name") as album_name,

# from json_parsed CROSS JOIN UNNEST(json_parsed.json_data) as jsn_line
# )

## Now enrich the playlist songs with the new features

`hybrid-vertex.spotify_train_3.unique_track_features` # need code

+

`hybrid-vertex.spotify_train_3.unique_artist_features` # need code

+

`hybrid-vertex.spotify_mpd.playlists_nested`

Trainv3-create-enriched-data
Fill missing

In [175]:
%%bigquery
CREATE OR REPLACE TABLE
  `hybrid-vertex.spotify_train_3.enriched_data` AS (
    SELECT
    a.* except(tracks),
      ARRAY(
    SELECT
      AS STRUCT CAST(track.pos AS int64) AS pos_can,
      case when track.artist_name = '' then 'NONE' else track.artist_name end AS artist_name_can,
      case when track.track_uri = '' then 'NONE' else track.track_uri  end AS track_uri_can,
      case when track.track_name = '' then 'NONE' else track.track_name  end AS track_name_can,
      case when track.album_uri = '' then 'NONE' else track.album_uri  end AS album_uri_can,
      case when track.artist_uri = '' then 'NONE' else track.artist_uri  end AS artist_uri_can,
      CAST(track.duration_ms AS float64) / 1.0 AS duration_ms_can,
      case when track.album_name = '' then 'NONE' else track.album_name end AS album_name_can,
      CAST(IFNULL(tf.track_pop, 0.0) as float64) / 1.0 AS track_pop_can,
      CAST(IFNULL(af.artist_pop, 0.0) as float64) / 1.0  AS artist_pop_can,
      case when af.artist_genres[OFFSET(0)] = '' then ['NONE'] else af.artist_genres end AS artist_genres_can,
      CAST(IFNULL(af.artist_followers, 0.0) as float64) / 1.0 AS artist_followers_can
    FROM
      UNNEST(tracks) as track
    INNER JOIN
      `hybrid-vertex.spotify_train_3.unique_track_features` AS tf --track features
    ON
      (track.track_uri = tf.track_uri)
    INNER JOIN
      `hybrid-vertex.spotify_train_3.unique_artist_features` AS af --artist features
      ON
      (track.artist_uri = af.artist_uri)
      ) AS tracks
  FROM 
  `hybrid-vertex.spotify_mpd.playlists_nested` as a)

Query complete after 0.00s: 100%|██████████| 37/37 [00:00<00:00, 18138.06query/s]                      


""


## Cross join + get rid of after-the-fact pos data in playlist

cross_join_songxplaylist_struct_query

`hybrid-vertex.spotify_train_3.ordered_position_training`

We create a data structure that creates unique song-playlist combos (every possible via cross-join). There is also a portion of pulling the last song in the playlist as the "seed track"

In [176]:
%%bigquery
CREATE OR REPLACE TABLE
  `hybrid-vertex.spotify_train_3.ordered_position_training` AS (
  WITH
    -- get every combination of song and its parent playlist
    unnest_cross AS(
    SELECT
      b.*,
      CONCAT(b.pid,"-",track.pos_can) AS pid_pos_id,
      CAST(track.pos_can AS int64) AS pos_can,
      track.artist_name_can ,
      track.track_uri_can ,
      track.album_uri_can,
      track.track_name_can ,
      track.artist_uri_can ,
      CAST(track.duration_ms_can AS float64) AS duration_ms_can,
      track.album_name_can ,
      track.track_pop_can ,
      track.artist_pop_can,
      track.artist_genres_can ,
      track.artist_followers_can 
    FROM (
      SELECT
        * EXCEPT(duration_ms)
      FROM
        `hybrid-vertex.spotify_train_3.enriched_data`) AS b
    CROSS JOIN
      UNNEST(tracks) AS track)
  SELECT
    a.* EXCEPT(tracks,
      num_tracks,
      num_artists,
      num_albums,
      num_followers,
      num_edits),
    ARRAY(
    SELECT
      AS STRUCT CAST(track.pos_can AS int64) AS pos_pl,
      track.artist_name_can AS artist_name_pl,
      track.track_uri_can AS track_uri_pl,
      track.track_name_can AS track_name_pl,
      track.album_uri_can AS album_uri_pl,
      track.artist_uri_can AS artist_uri_pl,
      CAST(track.duration_ms_can AS float64) AS duration_ms_pl,
      track.album_name_can AS album_name_pl,
      track.track_pop_can AS track_pop_pl,
      track.artist_pop_can AS artist_pop_pl,
      track.artist_genres_can AS artist_genres_pl,
      track.artist_followers_can AS artist_followers_pl,
    FROM
      UNNEST(tracks) AS track
    WHERE
      CAST(track.pos_can AS int64) < a.pos_can ORDER BY CAST(track.pos_can AS int64)) AS seed_playlist_tracks,
    ----- seed track part
    trx.pos_can AS pos_seed_track,
    trx.artist_name_can AS artist_name_seed_track,
    trx.artist_uri_can AS artist_uri_seed_track,
    trx.track_name_can AS track_name_seed_track,
    trx.track_uri_can AS track_uri_seed_track,
    trx.album_name_can AS album_name_seed_track,
    trx.album_uri_can AS album_uri_seed_track,
    trx.duration_ms_can AS duration_seed_track,
    trx.track_pop_can AS track_pop_seed_track,
    trx.artist_pop_can AS artist_pop_seed_track,
    trx.artist_genres_can as artist_genres_seed_track,
    trx.artist_followers_can as artist_followers_seed_track
  FROM
    unnest_cross AS a -- with statement
    ,
    UNNEST(a.tracks) AS trx
  WHERE
    CAST(trx.pos_can AS int64) = a.pos_can-1);

Query complete after 0.00s: 100%|██████████| 4/4 [00:00<00:00, 2018.43query/s]                        


""


## Update the playlist metadata with the new samples created above

Trainv3-clean-track-features

Get new metadata for the tracks now that there are updated track counts, durations, etc...

`hybrid-vertex.spotify_train_3.train`


In [177]:
from pprint import pprint
EPSILON = 1e-12
pprint(1+float(EPSILON))

1.000000000001


In [184]:
%%bigquery
create or replace table `hybrid-vertex.spotify_train_3.train` as (
WITH
  playlist_features_clean AS (
  SELECT
    pid_pos_id,
    SUM(trx.duration_ms_pl) / 1.0 AS duration_ms_seed_pl,
    COUNT(1) / 1.0 AS n_songs_pl,
    COUNT(DISTINCT trx.artist_name_pl) / 1.0 AS num_artists_pl,
    COUNT(DISTINCT trx.album_uri_pl) /1.0 AS num_albums_pl,
  FROM
    `hybrid-vertex.spotify_train_3.ordered_position_training`,
    UNNEST(seed_playlist_tracks) AS trx
  GROUP BY
    pid_pos_id)
    
SELECT
  a.* except(artist_genres_can, artist_genres_seed_track, track_pop_can, artist_pop_can, artist_followers_can,
            track_pop_seed_track, artist_pop_seed_track),
  b.* except(pid_pos_id),
  IFNULL(a.artist_genres_can[OFFSET(0)], "NONE") as artist_genres_can,
  IFNULL(a.artist_genres_seed_track[OFFSET(0)], "NONE") as artist_genres_seed_track,
  IFNULL(a.track_pop_can, 0.0) / 1.0 as  track_pop_can, 
  IFNULL(a.artist_pop_can, 0.0) / 1.0 as artist_pop_can,
  IFNULL(a.artist_followers_can, 0.0) / 1.0 as artist_followers_can,
  IFNULL(a.track_pop_seed_track, 0.0) / 1.0 as track_pop_seed_track,
  IFNULL(a.artist_pop_seed_track, 0.0) / 1.0 as artist_pop_seed_track,
  
FROM
  `hybrid-vertex.spotify_train_3.ordered_position_training` a
INNER JOIN
  playlist_features_clean b
ON
  a.pid_pos_id = b.pid_pos_id )

Query complete after 0.00s: 100%|██████████| 33/33 [00:00<00:00, 15764.47query/s]                      


""


## For TFRecords
Get rid of structs by creating new table with arrays from playlist_seed


In [185]:
%%bigquery
create or replace table `hybrid-vertex.spotify_train_3.train_flatten` as (
SELECT a.* except(seed_playlist_tracks, description),
    IFNULL(a.description, "") as description_pl,
    ARRAY(select t.pos_pl from UNNEST(seed_playlist_tracks) t) as pos_pl,
    ARRAY(select t.artist_name_pl from UNNEST(seed_playlist_tracks) t) as artist_name_pl,
    ARRAY(select t.track_uri_pl from UNNEST(seed_playlist_tracks) t) as track_uri_pl,
    ARRAY(select t.track_name_pl from UNNEST(seed_playlist_tracks) t) as track_name_pl,
    ARRAY(select t.duration_ms_pl from UNNEST(seed_playlist_tracks) t) as duration_ms_songs_pl,
    ARRAY(select t.album_name_pl from UNNEST(seed_playlist_tracks) t) as album_name_pl,
    ARRAY(select cast(t.artist_pop_pl as FLOAT64) from UNNEST(seed_playlist_tracks) t) as artist_pop_pl,
    ARRAY(select t.artist_followers_pl from UNNEST(seed_playlist_tracks) t) as artists_followers_pl,
    ARRAY(select case when t.track_pop_pl is null then 0. else t.track_pop_pl end from UNNEST(seed_playlist_tracks) t) as track_pop_pl,
    ARRAY(select t.artist_genres_pl[OFFSET(0)] from UNNEST(seed_playlist_tracks) t) as artist_genres_pl
    from `hybrid-vertex.spotify_train_3.train` a
)

Query complete after 0.00s: 100%|██████████| 3/3 [00:00<00:00, 1818.87query/s]                        


""


## We will use this table to export jsonl in the next notebook

Links on built in two tower and data [requriements](https://cloud.google.com/vertex-ai/docs/matching-engine/train-embeddings-two-tower#training_data)

In [186]:
%%bigquery
CREATE OR REPLACE TABLE
  `hybrid-vertex.spotify_train_3.train_json_export_table` AS (
  SELECT
    STRUCT(
        ARRAY(select artist_name_can) as artist_name_can,
        ARRAY(select track_uri_can) as track_uri_can,
        # album_uri_can,
        ARRAY(select track_name_can) as track_name_can,
        # artist_uri_can,
        ARRAY(select duration_ms_can) as duration_ms_can,
        ARRAY(select album_name_can) as album_name_can,
        ARRAY(select track_pop_can) as track_pop_can,
        ARRAY(select artist_pop_can) as artist_pop_can,
        ARRAY(select artist_followers_can) as artist_followers_can,
        ARRAY(select artist_genres_can) as artist_genres_can) AS candidate,
    STRUCT(
        # ARRAY(select pid_pos_id) as id_pl,
        ARRAY(select name) as name,
        ARRAY(select collaborative) as collaborative,
        ARRAY(select artist_name_seed_track) as artist_name_seed_track,
        # artist_uri_seed_track,
        ARRAY(select track_name_seed_track) as track_name_seed_track,
        # track_uri_seed_track,
        ARRAY(select album_name_seed_track) as album_name_seed_track,
        # album_uri_seed_track,
        ARRAY(select duration_seed_track) as duration_seed_track,
        ARRAY(select track_pop_seed_track) as track_pop_seed_track,
        ARRAY(select artist_pop_seed_track) as artist_pop_seed_track,
        ARRAY(select artist_followers_seed_track) as artist_followers_seed_track,
        ARRAY(select duration_ms_seed_pl) as duration_ms_seed_pl,
        ARRAY(select n_songs_pl) as n_songs_pl,
        ARRAY(select num_artists_pl) as num_artists_pl,
        ARRAY(select num_albums_pl) as num_albums_pl,
        ARRAY(select artist_genres_seed_track) as artist_genres_seed_track,
        ARRAY(select case when description_pl = '' then "NONE" else description_pl end) as description_pl,
        artist_name_pl,
        # track_uri_pl,
        track_name_pl,
        duration_ms_songs_pl,
        album_name_pl,
        artist_pop_pl as artist_pop_pl,
        artists_followers_pl as artists_followers_pl,
        track_pop_pl,
        artist_genres_pl
        ) AS query
  FROM
    `hybrid-vertex.spotify_train_3.train_flatten`)

Query complete after 0.00s: 100%|██████████| 3/3 [00:00<00:00, 1620.26query/s]                        


""


## create a smaller table for intital training

In [187]:
%%bigquery
CREATE OR REPLACE TABLE
  `hybrid-vertex.spotify_train_3.train_json_export_table_small` AS (
  SELECT
    STRUCT(
        ARRAY(select artist_name_can) as artist_name_can,
        ARRAY(select track_uri_can) as track_uri_can,
        # album_uri_can,
        ARRAY(select track_name_can) as track_name_can,
        # artist_uri_can,
        ARRAY(select duration_ms_can) as duration_ms_can,
        ARRAY(select album_name_can) as album_name_can,
        ARRAY(select track_pop_can) as track_pop_can,
        ARRAY(select artist_pop_can) as artist_pop_can,
        ARRAY(select artist_followers_can) as artist_followers_can,
        ARRAY(select artist_genres_can) as artist_genres_can) AS candidate,
    STRUCT(
        # ARRAY(select pid_pos_id) as id_pl,
        ARRAY(select name) as name,
        ARRAY(select collaborative) as collaborative,
        ARRAY(select artist_name_seed_track) as artist_name_seed_track,
        # artist_uri_seed_track,
        ARRAY(select track_name_seed_track) as track_name_seed_track,
        # track_uri_seed_track,
        ARRAY(select album_name_seed_track) as album_name_seed_track,
        # album_uri_seed_track,
        ARRAY(select duration_seed_track) as duration_seed_track,
        ARRAY(select track_pop_seed_track) as track_pop_seed_track,
        ARRAY(select artist_pop_seed_track) as artist_pop_seed_track,
        ARRAY(select artist_followers_seed_track) as artist_followers_seed_track,
        ARRAY(select duration_ms_seed_pl) as duration_ms_seed_pl,
        ARRAY(select n_songs_pl) as n_songs_pl,
        ARRAY(select num_artists_pl) as num_artists_pl,
        ARRAY(select num_albums_pl) as num_albums_pl,
        ARRAY(select artist_genres_seed_track) as artist_genres_seed_track,
        ARRAY(select case when description_pl = '' then "NONE" else description_pl end) as description_pl,
        artist_name_pl,
        # track_uri_pl,
        track_name_pl,
        duration_ms_songs_pl,
        album_name_pl,
        artist_pop_pl as artist_pop_pl,
        artists_followers_pl as artists_followers_pl,
        track_pop_pl,
        artist_genres_pl
        ) AS query
  FROM
    `hybrid-vertex.spotify_train_3.train_flatten` TABLESAMPLE SYSTEM (25 PERCENT))

Query complete after 0.00s: 100%|██████████| 2/2 [00:00<00:00, 1013.85query/s]                        


""


## Add unique song candidates for metric calcs later

In [ ]:
%%bigquery
CREATE OR REPLACE TABLE
  `hybrid-vertex.spotify_train_3.candidates_json` AS (
  select STRUCT(
        ARRAY(select artist_name_can) as artist_name_can,
        ARRAY(select track_uri_can) as track_uri_can,
        # album_uri_can,
        ARRAY(select track_name_can) as track_name_can,
        # artist_uri_can,
        ARRAY(select duration_ms_can) as duration_ms_can,
        ARRAY(select album_name_can) as album_name_can,
        ARRAY(select track_pop_can) as track_pop_can,
        ARRAY(select artist_pop_can) as artist_pop_can,
        ARRAY(select artist_followers_can) as artist_followers_can,
        ARRAY(select artist_genres_can) as artist_genres_can) as candidate
      from(
  SELECT DISTINCT
        artist_name_can,
        track_uri_can,
        # album_uri_can,
        track_name_can,
        # artist_uri_can,
        duration_ms_can,
        album_name_can,
        track_pop_can,
        artist_pop_can,
        artist_followers_can,
        artist_genres_can,
FROM
    `hybrid-vertex.spotify_train_3.train_flatten`))